In [ ]:
import torch 
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
os.chdir('../')
epoch_num = 20
batch_size = 128
lr = 1e-4
sigma = 1.0

In [ ]:
from sd.datasets import CelebADataset
from torch.utils.data import random_split

image_directory = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/'
full_dataset = CelebADataset(image_directory)
train_size = int(len(full_dataset) * 0.8)
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
from sd.train import VAE_Trainer

trainer = VAE_Trainer(lr, sigma)
trainer.train(train_loader, val_loader, epoch_num)
encoder = trainer.encoder
decoder = trainer.decoder


In [ ]:
from sd.dataseta import show_images
batch = next(iter(val_loader))
x = batch['image'][:5].to(trainer.device)
with torch.no_grad():
    n, c, h, w = x.shape
    noise = torch.randn(n, 4, h//8, w//8)
    z, _, _ = trainer.encoder(x, noise)
    x_hat = trainer.decoder(z)
show_images(x, 'Original Images')
show_images(x_hat, 'Re-generated Images')


In [ ]:
with torch.no_grad():
    z = torch.randn(n, 5 , h//8, w//8)
    generated_images = trainer.decoder(z)
show_images(genrated_images, "Generated_images")
